# SLTimer Tutorial 
This notebook shows you how to find the estimatation of a lens time delays from example light curve data using the PyCS code. For a detailed tutorial through the PyCS code, please visit this [address.](https://github.com/DarkEnergyScienceCollaboration/SLTimer/blob/master/notebooks/PyCS_Tutorial.ipynb)

## Getting Started

Let's first import `SLTimer`, as well as a few other important commands. 

In [ ]:
from __future__ import print_function
%matplotlib inline
from desc.sltimer import SLTimer
%load_ext autoreload
%autoreload 2

Next, let's start an `SLTimer` object, that we will use to carry out the measurement. 

In [ ]:
timer = SLTimer()

Now, let's download the `PyCS` demo1 trial data, testing it to make sure that we have downloaded the correct file.

In [ ]:
url = "https://raw.githubusercontent.com/COSMOGRAIL/PyCS/master/demo/demo1/data/trialcurves.txt"
datafile = S.download(url)

In [ ]:
timer.display_light_curve()

## Estimating Time Delays

Now we are ready to fit this data with a model. Let's start by having `SLTimer` use a `PyCS` "free-knot spline" model, without any microlensing.

In [ ]:
timer.create_basic_model()

We can also use polynomial models for the microlensing components, like this:

In [ ]:
timer.create_polynomial_model()

And we can improve this to a spline model for the microlensing (still with a spline model for the intrinsic variations):

In [ ]:
timer.create_spline_model()

The above options can also be accessed via the simple `estimate_time_delays` method, with an optional argument:

In [ ]:
timer.estimate_time_delays(method='pycs-spline')

## Estimating Uncertainties

Now we want to estimate the uncertainties on the time delays. `PyCS` does this by resampling the data, and re-optimizing. It's a lengthy process. First we need to make sure we don't have any old resampled data lying around, so we delete everything we can find. Watch out! This method is aggressive. Then, we do the error analysis in one shot. 

In [ ]:
timer.delete_old_files()

In [ ]:
timer.complete_error_analysis()

There you have it - a set of estimated time delays, assuming free-knot spline models for all variability components.